In [1]:
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine

In [3]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plot
import seaborn as sns
import numpy as np
import folium

In [124]:
with open('connection.txt', 'r') as f:
    DATABASE_URI = f.read()
engine = sal.create_engine(DATABASE_URI)
conn = engine.connect()

In [125]:
print(engine.table_names())

['companies', 'imoti']


# Properies with reduced price

In [126]:
query_reduced = "select * from most_reduced('АПАРТАМЕНТ', -5000, 100000)"
reduced = pd.read_sql_query(query_reduced, engine)

In [127]:
reduced.describe()

,lon,lat,price,price_sqm,area,floor,price_diff,price_diff_percentage
count,126.000000,126.000000,126.000000,126.000000,126.000000,123.000000,126.000000,126.000000
mean,42.683175,23.314936,76850.793651,957.108254,84.246032,3.528455,-11983.833333,-12.178968
std,0.029607,0.047248,17523.813224,225.252869,28.300583,2.704960,19764.774005,8.685115
min,42.615731,23.233606,26725.000000,450.440000,33.000000,1.000000,-210099.000000,-70.030000
25%,42.654809,23.277288,67192.500000,799.000000,68.000000,2.000000,-10710.000000,-12.795000
50%,42.685446,23.305564,78000.000000,914.125000,81.500000,3.000000,-8000.000000,-9.810000
75%,42.706436,23.357910,89975.000000,1116.420000,97.750000,4.000000,-6000.000000,-7.532500
max,42.738755,23.410840,99999.000000,1776.310000,222.000000,20.000000,-5001.000000,-5.090000


In [128]:
def set_anomaly_color(anomaly):
    # https://colorate.azurewebsites.net/Blend/0B7D7D_D6590D
    
    if anomaly == 1:
        return '#4DF78B'
    elif anomaly == 2:
        return '#47F141'
    elif anomaly == 3:
        return '#84EC35'
    elif anomaly == 4:
        return '#C7E62A'
    elif anomaly == 5:
        return '#E1B420'
    elif anomaly == 6:
        return '#DB5F16'
    elif anomaly == 7:
        return '#D60D13'
    

def get_bin(price_change, range_):
    # import pdb; pdb.set_trace()
    bin_size = (range_[1] - range_[0]) / 7
    
    iter_ = 3
    for i in range(1,7):
        if price_change >= range_[0] + bin_size * i:
            return iter_
        else:
            iter_ -= 1


reduced['bin'] = pd.qcut(reduced['price_diff'], q=7, precision=0, labels=list(np.arange(start=7, stop=0, step=-1)))
reduced = reduced[~pd.isnull(reduced['lon'])]

m = folium.Map(tiles='cartodbpositron', location=[reduced['lon'].mean(), reduced['lat'].mean()])

for ind, row in reduced.iterrows():
    folium.Circle(
        location=[row['lon'], row['lat']],
        #
        popup=str(row['price']) + ' EUR for ' + str(row['area']) + ' m2, PRICE_CHANGE: ' + str(row['price_diff']) + ', PRICE_REDUCTION_%: ' +  str(row['price_diff_percentage']) + ', link: ' + row['link'], # pop-up label for the marker
        radius=row['area']/3,
        color=set_anomaly_color(row['bin']) # set_color(row['price'], row['area']),
        ).add_to(m)

m

# Most recent entries

In [236]:
query_recent = "select * from get_prop_from_date('2020-05-09')"
recent = pd.read_sql_query(query_recent, engine)

In [237]:
### FLAG EVENTS
recent['tax_not_incl'] = recent['description'].map(str).apply(lambda x: re.search('(?:без|не е).{1,15}ДДС', x, re.IGNORECASE) is not None)
recent['investor'] = recent['description'].map(str).apply(lambda x: 'инвеститор' in x.lower())
recent['tanit_grup'] = recent['description'].map(str).apply(lambda x: 'танит груп' in x.lower())
recent['has_street'] = recent['description'].map(str).apply(lambda x: 'ул' in x.lower())
recent['is_subway'] = recent['description'].map(str).apply(lambda x: 'метро' in x.lower())
recent['has_view'] = recent['description'].map(str).apply(lambda x: 'гледка' in x.lower() or 'изглед' in x.lower())
recent['is_finished'] = recent['description'].map(str).apply(lambda x: re.search('акт(?:-|\s)?1[456]', x.lower()) is not None)
recent['is_under_construction'] = recent['description'].map(str).apply(lambda x: 'замазка' in x.lower() or 'шпакловка' in x.lower())
recent['is_new'] = recent['description'].map(str).apply(lambda x: 'новострояща' in x.lower()or 'затворен комплекс' in x.lower()) 

recent[['tax_not_incl', 'has_street', 'investor', 'tanit_grup', 'is_subway', 'has_view', 'is_finished', 'is_under_construction', 'is_new']].sum()

tax_not_incl              1320
has_street               13730
investor                  1623
tanit_grup                   3
is_subway                 7412
has_view                  2823
is_finished               4679
is_under_construction     4019
is_new                    2506
dtype: int64

In [238]:
#Show case
matches = recent['description'].map(str).apply(lambda x: re.search('((?:без|не е).{1,15}ДДС)', x, re.IGNORECASE) 
                                               if re.search('(?:без|не е).{1,15}ДДС', x, re.IGNORECASE) is not None 
                                               else None)
matches[~pd.isnull(matches)].apply(lambda x: x.group(1)).value_counts().head()

без ДДС             942
без включен ДДС     124
БЕЗ ДДС             103
без включено ДДС     63
без вкл. ДДС         26
Name: description, dtype: int64

In [239]:
def adjust_price_with_tax(df):
    tax_not_incl = df['description'].map(str).apply(lambda x: re.search('(?:без|не е).{1,15}ДДС', x, re.IGNORECASE) is not None)
    df['adjusted_price'] = df['price'] + df['price']/5
    df['adjusted_price_sqm'] = df['price_sqm'] + df['price_sqm']/5
    # print(df[tax_not_incl][['price', 'adjusted_price', 'price_sqm', 'adjusted_price_sqm']].head())
    df.loc[tax_not_incl,['price']] = df[tax_not_incl]['adjusted_price']
    df.loc[tax_not_incl,['price_sqm']] = df[tax_not_incl]['adjusted_price_sqm']
    # print(df[tax_not_incl][['price', 'adjusted_price', 'price_sqm', 'adjusted_price_sqm']].head())
    # print(df[~tax_not_incl][['price', 'adjusted_price', 'price_sqm', 'adjusted_price_sqm']].head())
    df.drop(columns=['adjusted_price', 'adjusted_price_sqm'], inplace=True)
    return df

In [240]:
# adjust price for TAXES not included
recent = adjust_price_with_tax(recent)

In [241]:
def get_anomalies(df):
    agg = df.groupby('place').agg({'price_sqm': ['std', 'mean', 'count']}).reset_index()
    agg.columns = ['_'.join(col).strip() for col in agg.columns.values]
    df = pd.merge(df, agg, left_on ='place', right_on ='place_', how='left')
    df['price_sqm_anomaly'] = round((df['price_sqm'] - df['price_sqm_mean']) / df['price_sqm_std'], 2)
    df['bin'] = pd.qcut(df['price_sqm_anomaly'], q=7, precision=0, labels=list(np.arange(start=7, stop=0, step=-1)))
    
    return df

recent = get_anomalies(recent)
print(recent.columns)
recent[['title', 'place', 'price_sqm_count', 'price_sqm_anomaly', 'bin',  'price_sqm_std', 'price_sqm', 'price_sqm_mean']].head()

Index(['id', 'title', 'address', 'details', 'place', 'region', 'lon', 'lat',
       'price', 'price_sqm', 'area', 'floor', 'views', 'measurement_day',
       'agency', 'date', 'description', 'link', 'tax_not_incl', 'investor',
       'tanit_grup', 'has_street', 'is_subway', 'has_view', 'is_finished',
       'is_under_construction', 'is_new', 'place_', 'price_sqm_std',
       'price_sqm_mean', 'price_sqm_count', 'price_sqm_anomaly', 'bin'],
      dtype='object')


,title,place,price_sqm_count,price_sqm_anomaly,bin,price_sqm_std,price_sqm,price_sqm_mean
0,ЕДНОСТАЕН АПАРТАМЕНТ,Люлин 8,143,0.12,3,258.480804,897.00,866.127832
1,ЧЕТИРИСТАЕН АПАРТАМЕНТ,Дианабад,331,1.48,1,383.992008,1739.49,1171.612369
2,ТРИСТАЕН АПАРТАМЕНТ,Горна баня,307,1.52,1,541.475670,1313.33,492.954404
3,ДВУСТАЕН АПАРТАМЕНТ,Хаджи Димитър,307,0.98,1,364.285093,1274.46,916.572886
4,ДВУСТАЕН АПАРТАМЕНТ,Обеля 2,167,0.51,2,390.521613,793.60,594.094850


In [200]:
import plotly.express as px
fig = px.scatter(recent[(recent['place'] == 'Младост 4') & (recent['title'].apply(lambda x: 'апартамент' in x.lower()))], 
                 x="price",
                 y="area", 
                 color="title",
                 hover_data=['link'])
fig.show()

In [201]:
fig_hist = px.histogram(recent[(recent['place'] == 'Младост 4') & (recent['title'].apply(lambda x: 'апартамент' in x.lower()))], 
                 x="price_sqm",
                 hover_data=['link'])
fig_hist.show()

In [202]:
#f, axes = plot.subplots(holmes['neighbourhood'].nunique()//3, 3, figsize=(25, 25))

def scatterplots_per_neighborhood():
    colors = {'ЕДНОСТАЕН АПАРТАМЕНТ':'red',
              'ДВУСТАЕН АПАРТАМЕНТ':'blue',
              'МЕЗОНЕТ':'green',
              'ЧЕТИРИСТАЕН АПАРТАМЕНТ':'black',
              'ТРИСТАЕН АПАРТАМЕНТ': 'orange',
              'МНОГОСТАЕН АПАРТАМЕНТ': 'yellow'}

    typs = ['ЕДНОСТАЕН АПАРТАМЕНТ', 'ДВУСТАЕН АПАРТАМЕНТ', 'МЕЗОНЕТ', 'ЧЕТИРИСТАЕН АПАРТАМЕНТ', 'ТРИСТАЕН АПАРТАМЕНТ', 'МНОГОСТАЕН АПАРТАМЕНТ']
    cols = ['red', 'blue', 'green', 'black', 'orange', 'pink']
    x_ticks = [0, 25000, 50000, 75000, 100000, 125000, 150000, 175000, 200000, 225000, 250000]
    y_ticks = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]



    for neigh in recent['place'].unique():
        #ind_x = ind // 3
        #ind_y = ind % 3
        plot.subplots(figsize=(10, 10))

        for ind, typ in enumerate(typs):

            holmes_subset = recent[(recent['place'] == neigh) & (recent['title'] == typ)]
            if holmes_subset.shape[0] > 0:
                plot.scatter(x=holmes_subset["price"], 
                     y=holmes_subset["area"],
                     label=typs[ind],
                     c=cols[ind])

        plot.legend(loc=4, prop={'size': 12})
        plot.title(neigh, fontsize=20)
        plot.xlabel('EUR', fontsize=20)
        plot.xticks(x_ticks, fontsize=12)
        plot.ylabel('sq_m', fontsize=20)
        plot.yticks(y_ticks, fontsize=12)
        plot.grid()
        plot.ylim(0, 250)
        plot.xlim(0, 250000)
        plot.savefig('output/' + neigh + '.png')
        plot.show()
        # plot.legend(handles = holmes_subset["title"].unique(), [colors[i] for i in holmes_subset["title"].unique()], loc=4)

        # plot.savefig(neigh +".png")

# scatterplots_per_neighborhood(recent)

In [203]:
# agg_df = recent.groupby(['neighbourhood', 'title']).agg({'price': ['count', 'median', 'mean', 'max', 'min'],
#                                       'price_sqm': ['median', 'mean', 'max', 'min'],
#                                       'area': ['median', 'min'],
#                                       'views': ['median', 'mean']}).reset_index()
# agg_df.columns = ['_'.join(col).strip() for col in df.columns.values]
# agg_df.sort_values(by='price_count', ascending=False, inplace=True)
# df.to_csv('holmes_agg.tsv', index=False, sep='\t')

In [204]:
# from shapely.geometry import Point, Polygon
# import geopy.distance
# import geopandas as gpd


# def convert_to_geopandas(df, lon_column='lon', lat_column='lat'):
#     crs = {'init': 'epsg:4326'}
#     df['coor'] = df[[lon_column, lat_column]].values.tolist()
#     df['coor'] = df['coor'].apply(Point)
#     df = gpd.GeoDataFrame(df,
#                           crs=crs,
#                           geometry='coor')

#     return df

# master_gdf = convert_to_geopandas(recent)

# Apartments

In [222]:
#Final filtering

def filter_plot(df):
    df = df[(df['title'].apply(lambda x: re.search('апартамент', x.lower()) is not None)) 
                    & (df['lon'] > 0) 
                    & (df['lat'] > 0)
                    & (df['price'] < 80000)]
    df['lon'] = df['lon'].astype(float)
    df['lat'] = df['lat'].astype(float)

plt_df = filter_plot(recent)

c:\users\zlatan angelov\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\zlatan angelov\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [223]:
def set_color(price, area):
    if price < 50 * 10**3:
        return '#28C223'
    elif price < 65 * 10**3:
        return '#FF3300'
    elif price < 80 * 10**3:
        return '#000000'
    elif price < 100 * 10**3:
        return '#19D6E0'
    else:
        return '#FF9900'
    


In [245]:
def plot_on_a_map(plt_df):
    m = folium.Map(tiles='cartodbpositron', location=[plt_df['lon'].mean(), plt_df['lat'].mean()])

    for ind, row in plt_df.iterrows():
        folium.Circle(
            location=[row['lon'], row['lat']],
            popup=str(row['price']) + ' EUR for ' + str(row['area']) + ' m2, EUR/sqm anomaly: ' + str(row['price_sqm_anomaly']) + ', link: ' + row['link'], # pop-up label for the marker
            radius=row['area']/3,
            color=set_anomaly_color(row['bin']) # set_color(row['price'], row['area']),
            ).add_to(m)

    return m
    
# plot_on_a_map(plt_df)

In [46]:
m.save(outfile='imoti_map.html')

# Parcels

In [246]:
# is_regulation varchar(10), is_water varchar(10), is_electricity varchar(10), below_area bigint, below_price bigint
query_parcels = "select * from get_parcels('ДА','ДА','ДА', 1000, 50000)"
parcels = pd.read_sql_query(query_parcels, engine)

In [247]:
parcels = get_anomalies(parcels)
plot_on_a_map(parcels)